In [1]:
import mysql.connector
import csv
import sys 
sys.path.append('..')
from utils.config import MYSQL_CONFIG

In [2]:
with mysql.connector.connect(**MYSQL_CONFIG) as conn:
    cursor = conn.cursor()
        
    # Execute the SHOW TABLES command
    cursor.execute("SHOW TABLES")
        
    # Iterate over the results 
    tables = cursor.fetchall()
    print(tables)

[('albums',), ('artists',), ('customers',), ('employees',), ('genres',), ('invoice_items',), ('invoices',), ('media_types',), ('playlist_track',), ('playlists',), ('tracks',)]


In [3]:
with mysql.connector.connect(**MYSQL_CONFIG) as conn:
    cursor = conn.cursor()
        
    # Execute the SHOW TABLES command
    cursor.execute("SHOW TABLES")
        
    # Iterate over the results 
    tables = [table[0] for table in cursor.fetchall()]
    print(tables)

['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


In [4]:
def get_table_names():
    with mysql.connector.connect(**MYSQL_CONFIG) as conn:
        cursor = conn.cursor()
        
        # Execute the SHOW TABLES command
        cursor.execute("SHOW TABLES")
        
        # Iterate over the results and return the first item in the tuple
        tables = [table[0] for table in cursor.fetchall()]
        
        return tables

get_table_names()

['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoice_items',
 'invoices',
 'media_types',
 'playlist_track',
 'playlists',
 'tracks']

In [5]:
def seed_table(table_name: str, csv_file_path: str) -> int:
    """
    Seeds the specified table with data from a CSV file.

    Args:
        table_name (str): The name of the table to seed.
        csv_file_path (str): The path to the CSV file containing the data.

    Returns:
        int: The number of rows inserted into the table.
    """
    with mysql.connector.connect(**MYSQL_CONFIG) as conn:
        cursor = conn.cursor()
        
        # Disable foreign key checks
        cursor.execute('SET foreign_key_checks = 0')
        
        # Load the data from the CSV file inot MySQL
        with open(csv_file_path) as f:
            csv_data = csv.reader(f)
            next(csv_data) # skip the header row
            rows_inserted = 0
            for row in csv_data:
                # replace the empty strings with None values
                row = [None if value == "" else value for value in row]
                place_holders = ', '.join(['%s']*len(row))
                query =f"""INSERT INTO {table_name} VALUES( {place_holders})"""
                try:
                    cursor.execute(query, row)
                    rows_inserted += 1
                except Exception as e:
                    print(e)
        
        # Commit the changes 
        conn.commit()
        cursor.execute('SET foreign_key_checks = 1')
        
    return rows_inserted

In [6]:
# Loop through each table and seed it with data from the correspoinding CSV file 
# table_names = get_table_names() # due to FK constraint need to seed the tables in this order
table_names = ['artists', 'albums', 'genres', 'media_types', 'tracks', \
    'employees', 'customers', 'invoices', 'invoice_items', 'playlists', 'playlist_track' ]
for table_name in table_names:
    csv_file_path = f"../output/{table_name.lower()}.csv"
    
    rows_inserted = seed_table(table_name, csv_file_path)
    print(f"{rows_inserted} rows inserted into {table_name} table.")
    

275 rows inserted into artists table.
347 rows inserted into albums table.
25 rows inserted into genres table.
5 rows inserted into media_types table.
3503 rows inserted into tracks table.
8 rows inserted into employees table.
59 rows inserted into customers table.
412 rows inserted into invoices table.
2240 rows inserted into invoice_items table.
18 rows inserted into playlists table.
8715 rows inserted into playlist_track table.
